## Building a Data Lake on AWS


### Introduction
In this workshop we are going to be get visualize sentiment of Yelp reviews from 2015 using a data set from the AWS Open Data Registry [Yelp Reviews NLP Fast.ai](https://registry.opendata.aws/fast-ai-nlp/). This will give you experience taking raw data and transforming it to provide new insights, optimize data sets in your data lake, and serverlessly visualize the results. We will start by uploading the Yelp review dataset and uploading it to S3, registering the raw data, and once registered transform the data to get only the columns necessary to run an NLP job on the reviews to get sentiment. We could build our own NLP model by leveraging [Amazon SageMaker](https://aws.amazon.com/sagemaker/), but in this workshop we will be using [Amazon Comprehend](https://aws.amazon.com/comprehend/) to get the sentiment of the reviews as an example of using the built-in APIs available from AWS. 

![Data Lake](../../docs/assets/images/yelp_dl.png)

### Goals of this workshop
1. Download review dataset from [Yelp Reviews NLP Fast.ai](https://registry.opendata.aws/fast-ai-nlp/).
2. Register raw data set as a table with the [AWS Glue Data Catalog](https://docs.aws.amazon.com/glue/latest/dg/tables-described.html).
3. Run a pyspark [AWS Glue Job] to convert data set into [Parquet](https://parquet.apache.org/) and get review sentiment with [Amazon Comprehend](https://aws.amazon.com/comprehend/).
4. Store transformed results in a new curated data set.
5. Serverless query the optimzied data set with [Amazon Athena](https://aws.amazon.com/athena/)
6. Provide visual insights of the results with [Amazon QuickSight](https://aws.amazon.com/quicksight/) or [Bokeh](https://bokeh.pydata.org/en/latest/)

This notebook is inspired by the blog [How to scale sentiment analysis using Amazon Comprehend, AWS Glue and Amazon Athena
](https://aws.amazon.com/blogs/machine-learning/how-to-scale-sentiment-analysis-using-amazon-comprehend-aws-glue-and-amazon-athena/)

In [ ]:
import boto3
import botocore
import json
import time
import os

import project_path
from lib import workshop

glue = boto3.client('glue')
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

session = boto3.session.Session()
region = session.region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

database_name = 'yelp' # AWS Glue Data Catalog Database Name
raw_table_name = 'raw_reviews' # AWS Glue Data Catalog raw table name
parquet_table_name = 'parq_reviews' # AWS Glue Data Catalog parquet table name
open_data_bucket = 'fast-ai-nlp'

### [Download Yelp Reviews](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-example-download-file.html) 

We will download the reviews from the Fast.ai NLP dataset available on the [AWS Open Data Registry](https://registry.opendata.aws/fast-ai-nlp/).

In [ ]:
try:
    s3.Bucket(open_data_bucket).download_file('yelp_review_full_csv.tgz', 'yelp_review_full_csv.tgz')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

### Untar Yelp Reviews

There are two `csv` files in the tarball. One is called `train.csv`, the other is `test.csv`. If we were to leverage this data set to build an new AI/ML model we would have cleaned and split data sets for both train and test. 

In [ ]:
!tar -xvzf yelp_review_full_csv.tgz

For those interested, The `readme.txt` file describes in more details the dataset.

In [ ]:
!cat yelp_review_full_csv/readme.txt

### View raw csv file

We will use [Pandas](https://pandas.pydata.org/) to read the csv and view the data set. You will notice the data contains 2 unnamed columns for rating and review. The rating is between 1-5 and the review is a free form text field.

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', -1)

df = pd.read_csv('yelp_review_full_csv/train.csv', header=None)
df.head(5)

### [Create S3 Bucket](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html)

We will create an S3 bucket that will be used throughout the workshop for storing our data.

[s3.create_bucket](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket) boto3 documentation

In [ ]:
bucket = workshop.create_bucket(region, session, 'datalake-')
print(bucket)

### [Upload to S3](https://docs.aws.amazon.com/AmazonS3/latest/dev/Welcome.html)

Next, we will upload the json file created above to S3 to be used later in the workshop.

[s3.upload_file](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.upload_file) boto3 documentation

In [ ]:
file_name = 'train.csv'
session.resource('s3').Bucket(bucket).Object(os.path.join('yelp', 'raw', file_name)).upload_file('yelp_review_full_csv/'+file_name)


### Discover the data in your Data Lake

In this next section we will be using [AWS Glue](https://aws.amazon.com/glue/) to discover, catalog, and transform your data. Glue currently only supports `Python 2.7`, hence we'll write the script in `Python 2.7`.

### Permission setup for invoking AWS Glue from this Notebook
In order to enable this Notebook to run AWS Glue jobs, we need to add one additional permission to the default execution role of this notebook. We will be using SageMaker Python SDK to retrieve the default execution role and then you have to go to [IAM Dashboard](https://console.aws.amazon.com/iam/home) to edit the Role to add AWS Glue specific permission. 

### Finding out the current execution role of the Notebook
We are using SageMaker Python SDK to retrieve the current role for this Notebook which needs to be enhanced to support the functionality in AWS Glue.

In [ ]:
# Import SageMaker Python SDK to get the Session and execution_role
import sagemaker
from sagemaker import get_execution_role
sess = sagemaker.Session()
role = get_execution_role()
role_name = role[role.rfind('/') + 1:]
print(role_name)

### Adding AWS Glue as an additional trusted entity to this role
This step is needed if you want to pass the execution role of this Notebook while calling Glue APIs as well without creating an additional **Role**. If you have not used AWS Glue before, then this step is mandatory. 

If you have used AWS Glue previously, then you should have an already existing role that can be used to invoke Glue APIs. In that case, you can pass that role while calling Glue (later in this notebook) and skip this next step.

On the IAM dashboard, please click on **Roles** on the left sidenav and search for this Role. Once the Role appears, click on the Role to go to its **Summary** page. Click on the **Trust relationships** tab on the **Summary** page to add AWS Glue as an additional trusted entity. 

Click on **Edit trust relationship** and replace the JSON with this JSON.
```
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": [
          "sagemaker.amazonaws.com",
          "glue.amazonaws.com"
        ]
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
```
Once this is complete, click on **Update Trust Policy** and you are done.

![IAM Roles](../../docs/assets/images/iam_roles_hl.png)

In [ ]:
print("https://console.aws.amazon.com/iam/home?region={0}#/roles/{1}".format(region, role_name))

### Create the [AWS Glue Catalog Database](https://docs.aws.amazon.com/glue/latest/dg/define-database.html)

When you define a table in the AWS Glue Data Catalog, you add it to a database. A database is used to organize tables in AWS Glue. You can organize your tables using a crawler or using the AWS Glue console. A table can be in only one database at a time.

There is a central Glue Catalog for each AWS account. When creating the database you will use your account id declared above as `account_id`

[glue.create_database](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.create_database)

In [ ]:
workshop.create_db(glue, account_id, database_name, 'Database for Yelp Reviews')

### [Create the Raw table in Glue](https://docs.aws.amazon.com/glue/latest/dg/tables-described.html)

When you define a table in AWS Glue, you also specify the value of a classification field that indicates the type and format of the data that's stored in that table. If a crawler creates the table, these classifications are determined by either a built-in classifier or a custom classifier. If you create a table manually in the console or by using an API, you specify the classification when you define the table. For more information about creating a table using the AWS Glue console, see [Working with Tables on the AWS Glue Console](https://docs.aws.amazon.com/glue/latest/dg/console-tables.html).

[glue.create_table](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.create_table)

In [ ]:
location = 's3://{0}/yelp/raw'.format(bucket)

response = glue.create_table(
    CatalogId=account_id,
    DatabaseName=database_name,
    TableInput={
        'Name': raw_table_name,
        'Description': 'Raw Yelp reviews dataset',
        'StorageDescriptor': {
            'Columns': [
                {
                    'Name': 'rating',
                    'Type': 'tinyint',
                    'Comment': 'Rating of from the Yelp review'
                },
                {
                    'Name': 'review',
                    'Type': 'string',
                    'Comment': 'Review text of from the Yelp review'
                }
            ],
            'Location': location,
            'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
            'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
            'SerdeInfo': {
                'SerializationLibrary': 'org.apache.hadoop.hive.serde2.OpenCSVSerde',
                'Parameters': {
                    'escapeChar': '\\',
                    'separatorChar': ',',
                    'serialization.format': '1'
                }
            },
        },
        'TableType': 'EXTERNAL_TABLE',
        'Parameters': {
            'classification': 'csv'
        }
    }
)

### View Yelp Raw Reviews Data 

To see the raw Yelp reviews we will be installing a python library for querying the data in the Glue Data Catalog with Athena. More information about [PyAthena](https://pypi.org/project/PyAthena/)

In [ ]:
!pip install PyAthena

In [ ]:
from pyathena import connect
from pyathena.util import as_pandas

cursor = connect(region_name=region, s3_staging_dir='s3://'+bucket+'/yelp/temp').cursor()
cursor.execute('select * from ' + database_name + '.' + raw_table_name + ' limit 10')

df = as_pandas(cursor)
df.head(5)

## Transform Raw data to provide insights and visualization

### [Detect Sentiment](https://docs.aws.amazon.com/comprehend/latest/dg/how-sentiment.html)
We are now going to transform the raw data using [PySpark](http://spark.apache.org/docs/latest/api/python/index.html) in an AWS Glue job to call Amazon Comprehend APIs to get sentiment analysis on the review, convert the data into parquet, and [partition](https://docs.aws.amazon.com/athena/latest/ug/partitions.html) by sentiment. This will allow us to optimize analytics queries when viewing data by sentiment and returning just the values we need leveraging the columnar format of parquet.

The example below is the API call required to pass text into the Comprehend API to detect sentiment and the result returned.

[client.detect_sentiment](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/comprehend.html#Comprehend.Client.detect_sentiment)

In [ ]:
import pprint as pp

client = boto3.client('comprehend', region_name=region)

response = client.detect_sentiment(Text='This API call is awesome!!! So easy to get sentiment of text!', LanguageCode='en')
pp.pprint(response['SentimentScore'])


### Uploading the code and other dependencies to S3 for AWS Glue
In order to run your code in AWS Glue, we need to upload the code and dependencies directly to S3 and pass those locations while invoking the Glue job. We will write the ETL job using Jupyter Notebooks cell magic [%%writefile](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile).

### Get Sentiment of Yelp Review Data

We will create a Pyspark job to add primary key and run a batch of reviews through [Amazon Comprehend](https://aws.amazon.com/comprehend/) to get sentiment analysis of the reviews. Replace `{region}` with the region this notebook is running in.

The job will limit the number of rows it converts due to timeliness of the workshop, but this code could be modified to run the entire data set.

The key points in this code is how easy it is to get access to the AWS Glue Data Catalog leveraging the [Glue libraries](https://github.com/awslabs/aws-glue-libs).

* [`glueContext.create_dynamic_frame.from_catalog`](https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-extensions-glue-context.html#aws-glue-api-crawler-pyspark-extensions-glue-context-create_dynamic_frame_from_catalog) - Read table metadata from the Glue Data Catalog using Glue libs to load tables into the job.
* `yelpDF = yelp.toDF()` - Easy conversion from [Glue DynamicFrame](https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-extensions-dynamic-frame.html) to [Spark DataFrame](https://spark.apache.org/docs/latest/sql-programming-guide.html) and vice-versa `joinedsink= DynamicFrame.fromDF(joinedDF, glueContext, "joined")`.
* Writing back S3 [`glueContext.write_dynamic_frame.from_options`](https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-extensions-glue-context.html#aws-glue-api-crawler-pyspark-extensions-glue-context-write_dynamic_frame_from_catalog) with options:
    * [Partition](https://docs.aws.amazon.com/athena/latest/ug/partitions.html) the data based on columns `connection_options = {"path": parquet_output_path, "partitionKeys": ["sentiment"]}`
    * Convert data to a [columnar format](https://docs.aws.amazon.com/athena/latest/ug/columnar-storage.html) `format="parquet"`

In [ ]:
%%writefile yelp_etl.py

import os
import sys
import boto3

from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame

import pyspark.sql.functions as F
from pyspark.sql import Row, Window, SparkSession
from pyspark.sql.types import *
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.functions import *

args = getResolvedOptions(sys.argv, ['JOB_NAME', 'S3_OUTPUT_BUCKET', 'S3_OUTPUT_KEY_PREFIX', 'DATABASE_NAME', 'TABLE_NAME', 'REGION'])

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# Covert Glue DynamocFrame to Spark DataFrame
yelp = glueContext.create_dynamic_frame.from_catalog(database=args['DATABASE_NAME'], table_name=args['TABLE_NAME'], transformation_ctx = "datasource0")
yelpDF = yelp.toDF().select('rating', 'review')

MIN_SENTENCE_LENGTH_IN_CHARS = 10 
MAX_SENTENCE_LENGTH_IN_CHARS = 4500
COMPREHEND_BATCH_SIZE = 25  ## This batch size results in groups no larger than 25 items
NUMBER_OF_BATCHES = 40
ROW_LIMIT = 10000 #Number of reviews we will process for this workshop

## Each task handles 25*40 records, there should be 10 partitions overall to process 10000 records.
ComprehendRow = Row("review", "rating", "sentiment")

def getBatchComprehend(input_list):
    arr = []
    bodies = [i[0] for i in input_list]
    client = boto3.client('comprehend',region_name=args['REGION'])

    def callApi(text_list):
        response = client.batch_detect_sentiment(TextList = text_list, LanguageCode = 'en')
        return response
  
    for i in range(NUMBER_OF_BATCHES):
        text_list = bodies[COMPREHEND_BATCH_SIZE * i : COMPREHEND_BATCH_SIZE * (i+1)]
        #response = client.batch_detect_sentiment(TextList = text_list, LanguageCode = 'en')
        response = callApi(text_list)
        for r in response['ResultList']:
            idx = COMPREHEND_BATCH_SIZE * i + r['Index']
            arr.append(ComprehendRow(input_list[idx][0], input_list[idx][1], r['Sentiment']))
  
    return arr

# Grab a sample set of records with review size under Comprehend limits
yelpDF = yelpDF \
  .withColumn('review_len', F.length('review')) \
  .filter(F.col('review_len') > MIN_SENTENCE_LENGTH_IN_CHARS) \
  .filter(F.col('review_len') < MAX_SENTENCE_LENGTH_IN_CHARS) \
  .limit(ROW_LIMIT)

record_count = yelpDF.count()
print('record count=' + str(record_count))

yelpDF = yelpDF.repartition(record_count/(NUMBER_OF_BATCHES*COMPREHEND_BATCH_SIZE))

 ## Concatenate submission id and body tuples into arrays of similar size
group_rdd = yelpDF.rdd.map(lambda l: (l.review.encode("utf-8"), l.rating)).glom()
  
transformed = group_rdd \
  .map(lambda l: getBatchComprehend(l)) \
  .flatMap(lambda x: x) \
  .toDF()

print("transformed count=" + str(transformed.count()))
      
transformedsink = DynamicFrame.fromDF(transformed, glueContext, "joined")
parquet_output_path = 's3://' + os.path.join(args['S3_OUTPUT_BUCKET'], args['S3_OUTPUT_KEY_PREFIX'])
print(parquet_output_path)
datasink5 = glueContext.write_dynamic_frame.from_options(frame = transformedsink, connection_type = "s3", connection_options = {"path": parquet_output_path, "partitionKeys": ["sentiment"]}, format="parquet", transformation_ctx="datasink5")
                                                                                                                      
job.commit()

### Upload the Yelp ETL script to S3
We will be uploading the `github_etl.py` script to S3 now so that Glue can use it to run the PySpark job. You can replace it with your own script if needed. If your code has multiple files, you need to zip those files and upload to S3 instead of uploading a single file like it's being done here.

In [ ]:
script_location = sess.upload_data(path='yelp_etl.py', bucket=bucket, key_prefix='yelp/codes')

In [ ]:
# Output location of the data. The input data will be split, transformed, and 
# uploaded to output/train and output/validation
s3_output_key_prefix = 'yelp/parquet/'

### Calling Glue APIs

Next we'll be creating Glue client via Boto so that we can invoke the `create_job` API of Glue. `create_job` API will create a job definition which can be used to execute your jobs in Glue. The job definition created here is mutable. While creating the job, we are also passing the code location as well as the dependencies location to Glue.

`AllocatedCapacity` parameter controls the hardware resources that Glue will use to execute this job. It is measures in units of `DPU`. For more information on `DPU`, please see [here](https://docs.aws.amazon.com/glue/latest/dg/add-job.html).

[glue.create_job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.create_job)

In [ ]:
from time import gmtime, strftime
import time

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

job_name = 'yelp-etl-' + timestamp_prefix
response = glue.create_job(
    Name=job_name,
    Description='PySpark job to extract Yelp review sentiment analysis',
    Role=role, # you can pass your existing AWS Glue role here if you have used Glue before
    ExecutionProperty={
        'MaxConcurrentRuns': 1
    },
    Command={
        'Name': 'glueetl',
        'ScriptLocation': script_location
    },
    DefaultArguments={
        '--job-language': 'python',
        '--job-bookmark-option': 'job-bookmark-disable'
    },
    AllocatedCapacity=5,
    Timeout=60,
)
glue_job_name = response['Name']
print(glue_job_name)

The aforementioned job will be executed now by calling `start_job_run` API. This API creates an immutable run/execution corresponding to the job definition created above. We will require the `job_run_id` for the particular job execution to check for status. We'll pass the data and model locations as part of the job execution parameters.

[glue.start_job_run](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.start_job_run)

In [ ]:
job_run_id = glue.start_job_run(JobName=job_name,
                                       Arguments = {
                                        '--S3_OUTPUT_BUCKET': bucket,
                                        '--S3_OUTPUT_KEY_PREFIX': s3_output_key_prefix,
                                        '--DATABASE_NAME': database_name,
                                        '--TABLE_NAME': raw_table_name,
                                        '--REGION': region
                                       })['JobRunId']
print(job_run_id)

### Checking Glue Job status

Now we will check for the job status to see if it has `SUCCEEDED`, `FAILED` or `STOPPED`. Once the job is succeeded, we have the transformed data into S3 in Parquet format which we will use to query with Athena and visualize with QuickSight. If the job fails, you can go to AWS Glue console, click on **Jobs** tab on the left, and from the page, click on this particular job and you will be able to find the CloudWatch logs (the link under **Logs**) link for these jobs which can help you to see what exactly went wrong in the job execution.

[glue.get_job_run](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.get_job_run)

In [ ]:
job_run_status = glue.get_job_run(JobName=job_name,RunId=job_run_id)['JobRun']['JobRunState']
while job_run_status not in ('FAILED', 'SUCCEEDED', 'STOPPED'):
    job_run_status = glue.get_job_run(JobName=job_name,RunId=job_run_id)['JobRun']['JobRunState']
    print (job_run_status)
    time.sleep(60)
print(job_run_status)

### Use a [Glue Crawler](https://docs.aws.amazon.com/glue/latest/dg/add-crawler.html) to Discover the transformed data

You can use a crawler to populate the AWS Glue Data Catalog with tables. This is the primary method used by most AWS Glue users. You add a crawler within your Data Catalog to traverse your data stores. The output of the crawler consists of one or more metadata tables that are defined in your Data Catalog. Extract, transform, and load (ETL) jobs that you define in AWS Glue use these metadata tables as sources and targets.

A crawler can crawl both file-based and table-based data stores. Crawlers can crawl the following data stores:

* Amazon Simple Storage Service (Amazon S3)
    * [Built-in Classifiers](https://docs.aws.amazon.com/glue/latest/dg/add-classifier.html#classifier-built-in)
    * [Custom Classifiers](https://docs.aws.amazon.com/glue/latest/dg/custom-classifier.html)
* Amazon Redshift
* Amazon Relational Database Service (Amazon RDS)
    * Amazon Aurora
    * MariaDB
    * Microsoft SQL Server
    * MySQL
    * Oracle
    * PostgreSQL
* Amazon DynamoDB
* Publicly accessible databases [Blog](https://aws.amazon.com/blogs/big-data/how-to-access-and-analyze-on-premises-data-stores-using-aws-glue/)
    * Aurora
    * MariaDB
    * SQL Server
    * MySQL
    * Oracle
    * PostgreSQL

[glue.create_crawler](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.create_crawler)

In [ ]:
parq_crawler_name = 'YelpCuratedCrawler'
parq_crawler_path = 's3://{0}/yelp/parquet/'.format(bucket)

response = glue.create_crawler(
    Name=parq_crawler_name,
    Role=role,
    DatabaseName=database_name,
    Description='Crawler for the Parquet Yelp Reviews with Sentiment',
    Targets={
        'S3Targets': [
            {
                'Path': parq_crawler_path
            }
        ]
    },
    SchemaChangePolicy={
        'UpdateBehavior': 'UPDATE_IN_DATABASE',
        'DeleteBehavior': 'DEPRECATE_IN_DATABASE'
    },
    TablePrefix='reviews_'
)

### Start the Glue Crawler

You can use a crawler to populate the AWS Glue Data Catalog with tables. This is the primary method used by most AWS Glue users. You add a crawler within your Data Catalog to traverse your data stores. The output of the crawler consists of one or more metadata tables that are defined in your Data Catalog. Extract, transform, and load (ETL) jobs that you define in AWS Glue use these metadata tables as sources and targets.

In [ ]:
response = glue.start_crawler(
    Name=parq_crawler_name
)

print ("Parquet Crawler: https://{0}.console.aws.amazon.com/glue/home?region={0}#crawler:name={1}".format(region, parq_crawler_name))

### Checking Glue crawler status

We will now monitor the crawler status waiting for it to get back into the `READY` state meaning the crawler completed it's crawl. You can also look at the [CloudWatch logs](https://docs.aws.amazon.com/glue/latest/dg/console-crawlers.html#console-crawlers-details) for the crawler for more details.

In [ ]:
crawler_status = glue.get_crawler(Name=parq_crawler_name)['Crawler']['State']
while crawler_status not in ('READY'):
    crawler_status = glue.get_crawler(Name=parq_crawler_name)['Crawler']['State']
    print(crawler_status)
    time.sleep(30)

### View transformed results

We will again us the PyAthena library to run queries against the newly created data set with sentiment result and in parquet format. In the interest of time, we will be using the [Bokeh](https://bokeh.pydata.org/en/latest/) within the notebook to visualize the results instead of [Amazon QuickSight](https://aws.amazon.com/quicksight/). QuickSight is able to use the same Athena queries to visualize the results as well as numerous [built-in connectors](https://docs.aws.amazon.com/quicksight/latest/user/supported-data-sources.html) to many datasources.

In [ ]:
cursor.execute('select rating, review, sentiment from yelp.reviews_parquet')

df = as_pandas(cursor)
df.head(10)

### Group  the data in the DataFrame by Sentiment

Using Pandas DataFrame functionality we will do the groupby locally. Alternatively we could have used the built-in [SQL and Aggregate functions](https://docs.aws.amazon.com/athena/latest/ug/functions-operators-reference-section.html) in Athena to achieve the same result.

In [ ]:
group = df.groupby(('sentiment'))
group.describe()

### Visualize results

The Bokeh framework has a number of built-in visualizations. For more information check out the [Bokeh Notebook](https://hub.mybinder.org/user/bokeh-bokeh-notebooks-6wusjpsu/notebooks/tutorial/00%20-%20Introduction%20and%20Setup.ipynb). In this example, we will be comparing the counts of Sentiment vs. Rating in the Yelp dataset using Bar Charts to display the results. Bokeh has great support for Jupyter Notebooks and provides the ability to display the output in notebook cells.

#### Visualize by Sentiment

In [ ]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.palettes import Spectral5
from bokeh.transform import factor_cmap
from bokeh.models import ColumnDataSource

output_notebook()

source = ColumnDataSource(group)
",".join(source.column_names)

sent_cmap = factor_cmap('sentiment', palette=Spectral5, factors=sorted(df.sentiment.unique()))

p = figure(plot_height=350, x_range=group)
p.vbar(x='sentiment', top='rating_count', width=1, line_color="white", 
       fill_color=sent_cmap, source=source)

p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Sentiment"
p.yaxis.axis_label = "Count"
p.y_range.start = 0

show(p)

#### Visualize by Rating

We will now compare what the Comprehend API came up with compared to the user rating in the data set. We are changing the group by in the dataframe to change the dataset.

In [ ]:
group = df.groupby(('rating'))
group.describe()

#### Visualize by User Rating

In [ ]:
source = ColumnDataSource(group)
",".join(source.column_names)

rating_cmap = factor_cmap('rating', palette=Spectral5, factors=sorted(df.rating.unique()))

p = figure(plot_height=350, x_range=group)
p.vbar(x='rating', top='review_count', width=1, line_color="white", 
       fill_color=rating_cmap, source=source)

p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Rating"
p.yaxis.axis_label = "Count"
p.y_range.start = 0

show(p)

### [Amazon QuickSight](https://aws.amazon.com/quicksight/) visualization

**Optional**
We can also visualize the data in [Amazon QuickSight](https://aws.amazon.com/quicksight/). You can follow allow with the [Getting Started](https://docs.aws.amazon.com/quicksight/latest/user/getting-started.html) guide for QuickSight to setup your account.

In [ ]:
print('https://{0}.quicksight.aws.amazon.com/sn/start?#'.format(region))

#### Manage Data sets

Once your account it created you will start by clicking the `Manage Data` button to create a new dataset that uses Amazon Athena to query the data. Click the `New data set` button and select the `Athena` data source, name the data source, and choose the `yelp` Glue database and `reviews_parquet` table. Finish creation by clicking the `Create data source` button. QuickSight supports a number of [Data Connectors](https://docs.aws.amazon.com/quicksight/latest/user/supported-data-sources.html). 

![QS Manage Data](../../docs/assets/images/qs_manage_data.png)

***

#### [Create New Data Set from S3](https://docs.aws.amazon.com/quicksight/latest/user/create-a-data-set-s3.html)

To create a data set using one or more text files (.csv, .tsv, .clf, or .elf) from Amazon S3, create a manifest that Amazon QuickSight can use to identify the files that you want to use, and also the upload settings needed to import them. When you create a data set using Amazon S3, the file data is automatically imported into [SPICE](https://docs.aws.amazon.com/quicksight/latest/user/welcome.html#spice).

![QS Manage Data](../../docs/assets/images/qs_new_dataset.png)

#### [Creating a Data Set Using Amazon Athena Data](https://docs.aws.amazon.com/quicksight/latest/user/create-a-data-set-athena.html)

You can connect to Amazon Athena data sources and use Athena data to create Amazon QuickSight data sets.

Before you try to read files from Amazon S3 buckets, make sure that you grant Amazon QuickSight access to them. For more information, see [Managing Amazon QuickSight Permissions to AWS Resources](https://docs.aws.amazon.com/quicksight/latest/user/managing-permissions.html).

In the `Data source name` textbox enter the name `yelp_reviews` and click `Create data source`.

![QS Manage Data](../../docs/assets/images/qs_athena_ds.png)

#### Chose the Glue table for Athena

Next, you will be selecting the `yelp` database we created in the Glue Data Catalog and the `reviews_parquet` table.

![QS Manage Data](../../docs/assets/images/qs_choose_table.png)

#### Edit the data set

![QS Edit Data Set](../../docs/assets/images/qs_data_edit.png)

#### Visualize!

![QS Edit Data Set](../../docs/assets/images/qs_visual.png)

## Cleanup

We will finish this workshop by removing the resources created in this notebook. Ensure each cell completes successfully to verify all resources have been removed.

In [ ]:
response = glue.delete_crawler(Name=parq_crawler_name)

In [ ]:
response = glue.delete_job(JobName=glue_job_name)

In [ ]:
response = glue.delete_database(
    CatalogId = account_id,
    Name = database_name
)

In [ ]:
workshop.delete_bucket_completely(bucket)